In [23]:
import pandas as pd
from utils.scraping_utils import get_html

In [24]:
rep_df = pd.read_csv('data\chosen_representatives.csv')
rep_df

,state,party,district,name,office_room,phone,committee,page_link
0,Alabama,D,7th,"Sewell, Terri",1035 LHOB,(202) 225-2665,Armed Services|House Administration|Joint Comm...,https://sewell.house.gov/
1,Alabama,R,1st,"Carl, Jerry",1330 LHOB,(202) 225-4931,Appropriations|Natural Resources,https://carl.house.gov
2,Alaska,D,At Large,"Peltola, Mary",153 CHOB,(202) 225-5765,Natural Resources|Transportation and Infrastru...,https://peltola.house.gov
3,American Samoa,R,Delegate,"Radewagen, Aumua Amata",2001 RHOB,(202) 225-8577,Foreign Affairs|Natural Resources|Veterans' Af...,https://radewagen.house.gov
4,Arizona,D,3rd,"Gallego, Ruben",1114 LHOB,(202) 225-4065,Armed Services|Natural Resources,https://rubengallego.house.gov/
...,...,...,...,...,...,...,...,...
80,Washington,R,4th,"Newhouse, Dan",504 CHOB,(202) 225-5816,Appropriations|Select Comm on the Strategic Co...,https://newhouse.house.gov
81,West Virginia,R,1st,"Miller, Carol",465 CHOB,(202) 225-3452,Ways and Means,https://miller.house.gov/
82,Wisconsin,D,2nd,"Pocan, Mark",1026 LHOB,(202) 225-2906,Appropriations,https://pocan.house.gov
83,Wisconsin,R,1st,"Steil, Bryan",1526 LHOB,(202) 225-3031,Financial Services|House Administration|Joint ...,https://steil.house.gov


In [25]:
def extract_issues_wrapper(rep_link, extract_issues):
    issues_link = rep_link + 'issues'
    issues_html = get_html(issues_link)
    if issues_html is None:
        return []
    
    issues = extract_issues(issues_html, rep_link)
    return issues

def extract_article_links_wrapper(base_link, issues, extract_article_links):
    article_links = []
    for issue in issues:
        issue_article_links = extract_article_links(issue, base_link)
        article_links.extend([{'issue': issue['name'], 'link': article_link} for article_link in issue_article_links])
    return article_links

def extract_articles_wrapper(article_links, extract_articles):
    articles = []
    for article_link in article_links:
        article = extract_articles(article_link)
        articles.append(article)
    return articles

def extract_articles(rep_link, extract_issues, extract_article_links, extract_article):
    if rep_link[-1] != '/':
        rep_link += '/'
    issues = extract_issues_wrapper(rep_link, extract_issues)
    issues = issues[:2]
    article_links = extract_article_links_wrapper(rep_link, issues, extract_article_links)
    articles = extract_articles_wrapper(article_links, extract_article)
    return articles

In [26]:
def sewell_extract_issues(issues_html, base_link):
    issues_list_element = issues_html.find(class_='issues-group')
    issue_elements = issues_list_element.find_all(class_='issues-group-item')
    issues = []
    for issue_element in issue_elements:
        issue = {
            'name': issue_element.text.strip(),
            'link': base_link + issue_element.a['href']
        }
        issues.append(issue)
    return issues

def sewell_extract_article_links(issue, base_link):
    articles_page = get_html(issue['link'])
    articles_links = []
    while True:
        articles_container = articles_page.find(class_='recordsContainer')
        article_groups = articles_container.find_all('table', class_='recordList')
        article_elements = []
        for article_group in article_groups:
            article_elements.extend(article_group.find('tbody').find_all('tr')) 

        for article_element in article_elements:
            cells = list(article_element.children)
            # date = cells[1].text.strip()
            # article_name = cells[3].text.strip()
            article_href = base_link + cells[3].a['href'].strip()
            articles_links.append(article_href)
        next_page_btn = articles_page.find('a', string='Next >')
        if next_page_btn is None or next_page_btn['href'] == '#':
            break
        next_page_link = base_link + next_page_btn['href']
        print(next_page_link)
        articles_page = get_html(next_page_link)

    return articles_links

def sewell_extract_article(article_link):
    return None
    article_page = get_html(article_link['link'])
    

In [27]:
extraction_functions = {
    'https://sewell.house.gov/': \
        {
            'extract_issues': sewell_extract_issues,
            'extract_article_links': sewell_extract_article_links,
            'extract_article': sewell_extract_article
        }
}

In [28]:
for link in rep_df['page_link']:
    if link in extraction_functions:
        page_functions = extraction_functions[link]
        print(link)
        articles = extract_articles(link, page_functions['extract_issues'], page_functions['extract_article_links'], page_functions['extract_article'])

https://sewell.house.gov/
https://sewell.house.gov//economy?page=2
https://sewell.house.gov//economy?page=3
https://sewell.house.gov//economy?page=4
https://sewell.house.gov//economy?page=5
https://sewell.house.gov//economy?page=6
https://sewell.house.gov//economy?page=7
https://sewell.house.gov//economy?page=8
https://sewell.house.gov//economy?page=9
https://sewell.house.gov//economy?page=10
https://sewell.house.gov//economy?page=11
https://sewell.house.gov//economy?page=12
https://sewell.house.gov//education?page=2
https://sewell.house.gov//education?page=3
https://sewell.house.gov//education?page=4
https://sewell.house.gov//education?page=5


InvalidSchema: No connection adapters were found for "{'issue': 'Economy', 'link': 'https://sewell.house.gov//2024/7/rep-sewell-announces-13th-annual-job-fair'}"

In [22]:
articles

[{'issue': 'Economy',
  'link': 'https://sewell.house.gov//2024/7/rep-sewell-announces-13th-annual-job-fair'},
 {'issue': 'Economy',
  'link': 'https://sewell.house.gov//2024/7/rep-sewell-announces-16-9-million-from-president-biden-s-bipartisan-infrastructure-law-to-upgrade-montgomery-bus-system'},
 {'issue': 'Economy',
  'link': 'https://sewell.house.gov//2024/6/rep-sewell-announces-14-million-in-department-of-transportation-raise-grants-for-the-cities-of-york-and-fairfield'},
 {'issue': 'Economy',
  'link': 'https://sewell.house.gov//2024/6/rep-sewell-hosts-seniors-resource-roundtable-in-linden'},
 {'issue': 'Economy',
  'link': 'https://sewell.house.gov//2024/5/rep-sewell-announces-16-9-million-in-epa-rebates-for-clean-school-buses-in-alabama'},
 {'issue': 'Economy',
  'link': 'https://sewell.house.gov//2024/5/rep-sewell-votes-for-faa-reauthorization-act-of-2024'},
 {'issue': 'Economy',
  'link': 'https://sewell.house.gov//2024/5/rep-sewell-honors-award-winning-small-businesses-in-a